In [1]:
import pandas as pd
import numpy as np 
import nltk
import string 
import re 
import warnings
import matplotlib.pyplot as plt


from datetime import datetime
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
# from sklearn.feature_extraction.text import CountVectorize


import pre


In [ ]:
path_0 = '../crawler/data/CarV4_1.csv'
path_1 = '../crawler/data/CarV4_0_0.csv'
path_2 = '../crawler/data/CarV4_0_1.csv'
path_3 = '../crawler/data/CarV4_0_2.csv'

df = pd.read_csv(path_0)
# df_1 = pd.read_csv(path_1)
# df = df_0.append(df_1, sort=False, ignore_index=True)


df.head() 

In [ ]:
df.info()

In [ ]:
ads = [('\\nA D V E R T I S E M E N T S\\n\\n\\t\\tgoogletag.cmd.push(function() { googletag.display(\'div-gpt-ad-1319716927473-1\'); });', ''),
       ('\\nA D V E R T I S E M E N T S\\n\\n\\t\\tgoogletag.cmd.push(function() { googletag.display(\'div-gpt-ad-1319716927473-2\'); });', ''),
       ('\\nA D V E R T I S E M E N T S\\n\\n\\t\\tgoogletag.cmd.push(function() { googletag.display(\'div-gpt-ad-1319716927473-3\'); });', ''),
       ('\\nA D V E R T I S E M E N T S\\n\\n\\t\\tgoogletag.cmd.push(function() { googletag.display(\'div-gpt-ad-1319716927473-4\'); });', ''),
       ('\\nA D V E R T I S E M E N T S\\n\\n\\t\\tgoogletag.cmd.push(function() { googletag.display(\'div-gpt-ad-1319716927473-5\'); });', ''),
       ('\\nA D V E R T I S E M E N T S\\n\\n\\t\\tgoogletag.cmd.push(function() { googletag.display(\'div-gpt-ad-1319716927473-6\'); });', ''),
       ('\\nA D V E R T I S E M E N T S\\n\\n\\t\\tgoogletag.cmd.push(function() { googletag.display(\'div-gpt-ad-1319716927473-7\'); });', ''),
       ('\\nA D V E R T I S E M E N T S\\n\\n\\t\\tgoogletag.cmd.push(function() { googletag.display(\'div-gpt-ad-1319716927473-8\'); });', ''),
       ('\\nA D V E R T I S E M E N T S\\n\\n\\t\\tgoogletag.cmd.push(function() { googletag.display(\'div-gpt-ad-1319716927473-9\'); });', ''),
       ('\\nA D V E R T I S E M E N T S\\n\\n\\t\\tgoogletag.cmd.push(function() { googletag.display(\\\'div-gpt-ad-1319716927473-1\\\'); });', ''),
       ('\\nA D V E R T I S E M E N T S\\n\\n\\t\\tgoogletag.cmd.push(function() { googletag.display(\\\'div-gpt-ad-1319716927473-2\\\'); });', ''),
       ('\\nA D V E R T I S E M E N T S\\n\\n\\t\\tgoogletag.cmd.push(function() { googletag.display(\\\'div-gpt-ad-1319716927473-3\\\'); });', ''),
       ('\\nA D V E R T I S E M E N T S\\n\\n\\t\\tgoogletag.cmd.push(function() { googletag.display(\\\'div-gpt-ad-1319716927473-4\\\'); });', ''),
       ('\\nA D V E R T I S E M E N T S\\n\\n\\t\\tgoogletag.cmd.push(function() { googletag.display(\\\'div-gpt-ad-1319716927473-5\\\'); });', ''),
       ('\\nA D V E R T I S E M E N T S\\n\\n\\t\\tgoogletag.cmd.push(function() { googletag.display(\\\'div-gpt-ad-1319716927473-6\\\'); });', ''),
       ('\\nA D V E R T I S E M E N T S\\n\\n\\t\\tgoogletag.cmd.push(function() { googletag.display(\\\'div-gpt-ad-1319716927473-7\\\'); });', ''),
       ('\\nA D V E R T I S E M E N T S\\n\\n\\t\\tgoogletag.cmd.push(function() { googletag.display(\\\'div-gpt-ad-1319716927473-8\\\'); });', ''),
       ('\\nA D V E R T I S E M E N T S\\n\\n\\t\\tgoogletag.cmd.push(function() { googletag.display(\\\'div-gpt-ad-1319716927473-9\\\'); });', ''),
       ('\"', '\''),
       ('\", \"\\n', '\', \'\\n'),  # use for split person
       ('[', ''),
       (']', ''),
       ############### Before personal info ########
       ('\\n\\t\\t\\t\\t', ''),
       ('\\n\\n-', ''),
       ('\\n\\t\\t\\t-', ''),
       #############################################
       ############## Begin of a comment ###########
       ('\\n\\t\\n\\n', ''),
       ('\\n\\t\\t\\t', ''),
       ('\\t\\t\\t', '')
       #############################################
      ]

len(df.columns[3:])


# df['to_bog'] = df['transmission'] + df['suspension'] + df['windows_windshield'] + df['fuel_system'] + df['body_paint'] + df['clutch'] + \
#                 df['seat_belts_air_bags'] + df['wheels_hubs'] + df['cooling_system'] + df['brakes'] + df['electrical'] + df['lights'] + \
#                 df['steering'] + df['drivetrain'] + df['accessories-interior'] + df['accessories-exterior'] + df['miscellaneous'] + \
#                  df['exhaust_system'] + df['AC_heater'] + df['engine']


In [ ]:
# remove ads in cells
for cols in df.columns:
    for k, v in ads:
        df[cols] = df[cols].apply(lambda x : str(x).replace(k, v))

# split data based on different person
for i in range(3, len(df.columns)):
        df[df.columns[i]] = df[df.columns[i]].apply(lambda x : str(x).split('\', \'') if x != '0' else 0)

df.head()

In [ ]:
columns = ['Make', 'Model', 'Year', 'Comments', 'Label']
df_comment = pd.DataFrame(columns=columns)

# split each cmment to a row and add label save into a new DF
rows = df.shape[0]
cols = df.shape[1]

for row_index in range(rows):
    for col_index in range(3, cols):        
        cell = df.iloc[row_index][col_index]
        if cell != 0:
            for comment in cell:
                label = df.columns[col_index]
                line = {'Make':df.iloc[row_index][0], 'Model':df.iloc[row_index][1], 'Year':df.iloc[row_index][2], 'Comments':comment,\
                       'Label':label}     # Make model year
                df_comment = df_comment.append(line, ignore_index=True)   # df.append will generate a new d



### check point 1


In [ ]:
# df_comment.head()
# df_comment.to_csv(path_or_buf='0_individual_comments/indiv_comment3.csv', index=False)

In [ ]:
df_indiv = pd.read_csv('0_individual_comments/indiv_comment3.csv')
df_indiv.head()

In [ ]:
warnings.filterwarnings("ignore")

date_list = []
exp_list = []       # capture exception index
comment_list = []

print('total lines: {}'.format(df_indiv.shape[0]))

for i in range(df_indiv.shape[0]):
    date_ = df_indiv.iloc[i][3].split('\\n\\n')[-1].replace('\'', '').replace('(reported on)','').strip()
    date_ = date_[:-4] + ' ' + date_[-4:]
    comment = df_indiv.iloc[i][3].split('\\n\\n')[:-1]        # New comment without date but location
    
    try:
        datetime_object = datetime.strptime(date_, '%b %d %Y')
    except:
        date_list.append('EXP')
        exp_list.append(i)
        comment_list.append(' ')
    else:
        date_list.append(date_)
        comment_list.append(comment)

# print(date_list[:10])

df_indiv['Date'] = date_list
df_indiv['Comments'] = comment_list

# del don't have correct date format
df_indiv = df_indiv[df_indiv.Date != 'EXP']

print('Lines after del:{}'.format(df_indiv.shape[0]))

### Check Point 2


In [ ]:
# df_indiv.to_csv(path_or_buf='1_date_added/df_with_date3.csv', index=False)

In [ ]:
df_indi_date = pd.read_csv('1_date_added/df_with_date3.csv')
df_indi_date.head()

In [ ]:
countries = []
states = []
comments = []

for row_index in range(df_indi_date.shape[0]):
    processed = pre.string_processor(df_indi_date.iloc[row_index][3])
    countries.append(processed.split()[-1])
    states.append(processed.split()[-2])
    comments.append(' '.join(processed.split()[:-3]))
    
print(len(countries), len(countries), len(countries))

In [ ]:
df_indi_date['Comments'] = comments
df_indi_date['country'] = countries
df_indi_date['state'] = states

df_indi_date.head()

### Check Point 3


In [ ]:
# df_indi_date.to_csv(path_or_buf='2_location_added/df_with_loc3.csv', index=False)

In [ ]:
df_final = pd.read_csv('2_location_added/df_with_loc3.csv')

df_toyota = df_final[df_final['Make'] == 'Toyota']

date = []
for i in range(df_toyota.shape[0]):
    m_y = ' '.join([df_toyota.iloc[i][5].split()[0], df_toyota.iloc[i][5].split()[-1]])
    date.append(datetime.strptime(m_y, '%b %Y').date())

df_toyota['Date'] = date

df_toyota.head()

In [ ]:
first_day = datetime.strptime('Jan 2002', '%b %Y').date()

df_sample = df_toyota[df_toyota['Date'] >= first_day]

date_table = {}
for date in set(df_sample['Date']):
    date_table[date] = 0

for i in range(df_sample.shape[0]):
    date_table[df_sample.iloc[i][5]] += 1
    
# date_table

In [ ]:
def plot_dict(dict_):
    plt.figure(figsize=(60, 20))
    plt.title('Toyota', fontsize=50)
    plt.bar(range(len(dict_)), list(dict_.values()))
    plt.xticks(range(len(dict_)), list(dict_.keys()),rotation='vertical')
    plt.ylabel('# of complains')
    plt.xlabel('Date')

plot_dict(date_table)